In [8]:
library(tidyverse)
library(readr)
library(tidymodels)

In [32]:
import_ad_data <- read_csv("Data/ad.data.csv", col_names = FALSE, col_select = c("X1", "X2", "X3", "X1560", "X1559"), n_max = 1000)
ad_data <- rename(import_ad_data, Height = X1, Width = X2, Ratio = X3, Attributes = X1560, Type = X1559) |>
           filter(Height != "?" , Width != "?" , Ratio != "?") |>
           arrange (Attributes)

total_type <- ad_data |>
              group_by(Type) |>
              summarize(n = n())
total_type

type_factor <- ad_data |>
               mutate(Type = as_factor(Type))
type_factor


Rows: 1000 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): X1, X2, X3, X1559
dbl (1): X1560

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Type,n
<chr>,<int>
ad.,386
nonad.,370


Height,Width,Ratio,Attributes,Type
<chr>,<chr>,<chr>,<dbl>,<fct>
88,520,5.909,0,ad.
51,433,8.4901,0,ad.
60,468,7.8,0,nonad.
54,239,4.4259,0,nonad.
116,200,1.7241,0,nonad.
40,40,1,0,nonad.
31,96,3.0967,1,ad.
60,468,7.8,1,ad.
60,468,7.8,1,ad.
